In [ ]:
from marvin.tools.query import Query
from marvin.tools.maps import Maps
import numpy as np
from sympy import *

In [ ]:
tot = [] #All galaxies
agns = [] #AGNs

In [ ]:
def get_masked(maps, emline):
    """Function to get masked data arrays without bad snr spaxels

    Parameters:
    maps - Marvin Maps object of the galaxy
    emline - String of desired quantity (e.g. 'emline_gflux_ha_6564')

    Returns:
    Data array with masked values if the spaxel has an snr < 5
    """    
    data_array = maps[emline]
    data_array_masked = data_array.masked

    #Masks all spaxels that don't reach the cutoff SNR
    data_array_masked.mask |= maps.bin_snr.value < 5
    
    return data_array_masked

In [ ]:
def get_trues(arr):
    """Function to count the number of True bools in a data array

    Parameters:
    arr - Data array containing boolean values

    Returns:
    Number of True values in arr
    """
    counter = 0
    for elem in arr.compressed():
        if elem:
            counter += 1
    return counter

In [ ]:
def get_1d_array(arr, arr_bools):
    """Creates numpy array containing a subset of the values in the data array

    Parameters:
    arr - Data array
    arr_bools - Corresponding array containing bools which specify whether
                each value is part of the subset being returned

    Returns:
    numpy array containing the subset of values
    """     
    lst = []
    for i in range(len(arr)):
        for j in range(len(arr)):
            if arr_bools[i][j] and not np.ma.is_masked(arr[i][j]):
                lst.append(arr[i][j])
    return np.array(lst)

In [ ]:
def get_mean_20th(arr):
    """Calculate the mean of the top 20% of values in an array

    Parameters:
    arr - numpy array containing floats

    Returns:
    Mean of the top 20% of values in an array
    """    
    num_20 = round(len(arr)*0.2)
    arr_20 = arr[np.argsort(arr)[-num_20:]]
    
    return np.mean(arr_20)

In [ ]:
def get_ew_val(ew, arr_bools):
    """Function to calculate the mean of the top 20% of the ews for
    a specific subset of the spaxels

    Parameters:
    arr - Data array of ews
    arr_bools - Corresponding array containing bools which specify whether
                each spaxel is a part of the subset of ews being calculated

    Returns:
    Mean of the top 20% of the subset of ews
    """
    ewhas = get_1d_array(ew, arr_bools)
    
    return get_mean_20th(ewhas)

In [ ]:
def combine_bool_arr(arr_1, arr_2):
    """Combines data arrays containing booleans according to "or" logic

    Parameters:
    arr_1 - First data array containing bools
    arr_2 - Second data array containing bools

    Returns:
    Returns new array which contains True in each position that either 
    arr_1 or arr_2 contains True
    """      
    arr_1_copy = arr_1.copy()
    
    for i in range(len(arr_2)):
        for j in range(len(arr_2)):
            if arr_2[i][j]:
                arr_1_copy.mask[i][j] = False
                arr_1_copy[i][j] = True

    return arr_1_copy

In [ ]:
def get_sb_val(ha, arr_bools):
    """Calculates the mean of the top 20% of the surface brightnesses
    for a subset of spaxels

    Parameters:
    arr - Data array of ha emissions
    arr_bools - Corresponding array containing bools which specify whether each 
                spaxel is a part of the subset being used for the calculation

    Returns:
    Mean of the top 20% of the surface brightnesses for a subset of spaxels
    """
    ha_1d = 9.5214e42*get_1d_array(ha, arr_bools)*1e-17 #flux in erg/s/kpc^2
    sb = 4*np.pi*ha_1d/(2.4241e-6**2)
    
    return np.log10(np.mean(sb))

In [ ]:
def get_d_bpt_i(x_0, y_0):
    """Calculates d_bpt_i (the distance from the point (x_0, y_0) to the demarcation line)

    Parameters:
    x_0 - log(sii/ha) value
    y_0 - log(oiii/hb) value

    Returns:
    Distance from the point (x_0, y_0) to the demarcation line
    """    
    a = Symbol('a')
    a_val = solve(a**4 + (0.32 - x_0)*a**3  + (0.72*y_0 - 0.936)*a - 0.5184, a)[0]
    x_1 = a_val + 0.32
    
    d2 = (x_1 - x_0)**2 + (0.72/(x_1 - 0.32) + 1.30 - y_0)**2
    
    return sqrt(d2)

In [ ]:
def get_d_bpt(arr_x_0, arr_y_0, arr_bools):
    """Calculates d_bpt (mean of top 20% of b_bpt_i values)

    Parameters:
    arr_x_0 - Data array of log(sii/ha) values
    arr_y_0 - Data array of log(oiii/hb) values
    arr_bools - Corresponding array containing bools which specify whether each 
                spaxel is a part of the subset being used for the calculation

    Returns:
    Mean of top 20% of b_bpt_i values
    """
    d_bpts = []
    
    for i in range(len(arr_bools)):
        for j in range(len(arr_bools)):
            if arr_bools[i][j] and not np.ma.is_masked(arr_x_0[i][j]) and not np.ma.is_masked(arr_y_0[i][j]):
                d_bpts.append(get_d_bpt_i(arr_x_0[i][j], arr_y_0[i][j]))
         
    return get_mean_20th(np.array(d_bpts))

In [ ]:
def classify_gal(gal_id):
    """Uses Wylezalek et al. 2017's optimized criteria to determine whether
    a galaxy hosts an AGN (Wylezalek et al. 2017)

    Parameters:
    gal_id - MaNGA id of the galaxy

    Returns:
    Function has no return value, however, appends galaxies that
    fulfill the criteria to the list of AGNs
    """    
    try:
        maps = Maps(mangaid=gal_id)
    except:
        return None 
    
    if 28 in maps.quality_flag.bits or 29 in maps.quality_flag.bits or 30 in maps.quality_flag.bits:
        print('Critical Flag!\n')
        return None
    
    ha = get_masked(maps, 'emline_gflux_ha_6564') 
    nii = get_masked(maps, 'emline_gflux_nii_6585')
    hb = get_masked(maps, 'emline_gflux_hb_4862')
    oiii = oiii = get_masked(maps, 'emline_gflux_oiii_5008')
    
    sii_6718 = get_masked(maps, 'emline_gflux_sii_6718')
    sii_6732 = get_masked(maps, 'emline_gflux_sii_6732')
    sii = sii_6718 + sii_6732
    
    nii_count = nii.count()
    sii_count = sii.count()
    
    log_nii_ha = np.log10(nii/ha)
    log_oiii_hb = np.log10(oiii/hb)
    log_sii_ha = np.log10((sii_6718 + sii_6732)/ha)
    
    nii_agn_bools = (0.61/(log_nii_ha - 0.47) + 1.19 < log_oiii_hb) | (log_nii_ha > 0.47)             
    nii_agn = get_trues(nii_agn_bools)
    
    nii_comp_bools = (0.61/(log_nii_ha - 0.47) + 1.19 > log_oiii_hb) & (0.61/(log_nii_ha - 0.05) + 1.3 < log_oiii_hb) & \
                     (log_nii_ha < 0.05)
    nii_comp = get_trues(nii_comp_bools)
    
    sii_agn_bools = ((0.72/(log_sii_ha - 0.32) + 1.30 < log_oiii_hb) | (log_sii_ha > 0.32)) & \
                    (1.89*log_sii_ha + 0.76 < log_oiii_hb)
    sii_agn = get_trues(sii_agn_bools)
    
    sii_liner_bools = ((0.72/(log_sii_ha - 0.32) + 1.30 < log_oiii_hb) | (log_sii_ha > 0.32)) & \
                      (1.89*log_sii_ha + 0.76 > log_oiii_hb)
    sii_liner = get_trues(sii_liner_bools)
    
    f_A_N = (80*nii_agn + 20*nii_comp)/nii_count
    f_A_S = 100*sii_agn/sii_count
    f_L_S = 100*sii_liner/sii_count
    f_AL_S = f_A_N + f_L_S
    
    ewha = get_masked(maps, 'emline_sew_ha_6564')
    
    nii_agn_comp_bools = combine_bool_arr(nii_agn_bools, nii_comp_bools)
    ewha_A_N = get_ew_val(ewha, nii_agn_comp_bools)
    ewha_A_S = get_ew_val(ewha, sii_agn_bools)
    ewha_L_S = get_ew_val(ewha, sii_liner_bools)
    ewha_AL_S = (ewha_A_S + ewha_L_S)/2
    
    if f_A_N > 10 and ewha_A_N > 5 and f_AL_S > 15 and ewha_AL_S > 5:
        sii_agn_liner_bools = combine_bool_arr(sii_agn_bools, sii_liner_bools)
        sb_A_N = get_sb_val(ha, nii_agn_comp_bools)
        sb_AL_S = get_sb_val(ha, sii_agn_liner_bools)
        
        if sb_A_N > 37.5 or sb_AL_S > 37.5:
            d_bpt = get_d_bpt(log_sii_ha, log_oiii_hb, sii_agn_liner_bools)
            
            if d_bpt > 0.3:
                agns.append(gal_id)

In [ ]:
q = Query(limit=10000)
r = q.run()

for i in range(r.totalcount):
    tot.append(r.results[i].mangaid)
    classify_gal(r.results[i].mangaid)

In [ ]:
print(len(tot))

In [ ]:
print(tot)

In [ ]:
print(len(agns))

In [ ]:
print(agns)

In [ ]:
with open('AGN Wylezalek Optimized MaNGAids.txt', 'w') as f:
    for gal_id in agns:
        f.write("%s\n" % gal_id)